Banking Customer Churn Prediction

In [92]:
# import warnings
# warnings.filterwarnings('ignore')

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619.0,France,Female,42,2,0.00,1,1,1,101348.88,1.0
1,2,15647311,Hill,608.0,Spain,Female,41,1,83807.86,1,0,1,112542.58,0.0
2,3,15619304,Onio,502.0,France,Female,42,8,159660.80,3,1,0,113931.57,1.0
3,4,15701354,Boni,699.0,France,Female,39,1,0.00,2,0,0,93826.63,0.0
4,5,15737888,Mitchell,850.0,Spain,Female,43,2,125510.82,1,1,1,79084.10,0.0


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      9995 non-null   float64
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  9999 non-null   float64
 13  Exited           9904 non-null   float64
dtypes: float64(4), int64(7), object(3)
memory usage: 1.1+ MB



**data Cleaning & Preprocessing**
---

In [95]:
df.isnull().sum()

RowNumber           0
CustomerId          0
Surname             0
CreditScore         5
Geography           0
Gender              0
Age                 0
Tenure              0
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary     1
Exited             96
dtype: int64

In [96]:
df.dropna(subset='Exited', inplace=True)

In [97]:
df[df['CreditScore'].isnull()]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
433,434,15595039,Manna,NaN,Germany,Female,37,8,114754.08,1,1,0,136050.44,1.0
435,436,15581197,Ricci,NaN,France,Female,51,3,99286.98,1,0,1,85578.63,0.0


In [98]:
df[df['EstimatedSalary'].isnull()]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
458,459,15707362,Yin,514.0,Germany,Male,43,1,95556.31,1,0,1,NaN,1.0


In [99]:
features = [
    'CreditScore', 
    'EstimatedSalary',
    'Age',
    'Tenure',
    'Balance',
    'NumOfProducts',
    'HasCrCard',
    'IsActiveMember'
]

imputer = KNNImputer(n_neighbors=5)
imputed_values = imputer.fit_transform(df[features])

df_imputed = pd.DataFrame(imputed_values, columns=features, index=df.index)
df[features] = df_imputed

In [100]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

***Analysis***
---

In [101]:
# Understanding the target
df['Exited'].value_counts()

Exited
0.0    7883
1.0    2021
Name: count, dtype: int64

In [102]:
df['HasCrCard'].value_counts()

HasCrCard
1.0    6983
0.0    2921
Name: count, dtype: int64

In [103]:
df['IsActiveMember'].value_counts()

IsActiveMember
1.0    5096
0.0    4808
Name: count, dtype: int64

In [104]:
df['NumOfProducts'].value_counts()

NumOfProducts
1.0    5041
2.0    4541
3.0     262
4.0      60
Name: count, dtype: int64

In [105]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
RowNumber,9904.0,5.001587e+03,2888.258558,1.00,2.501750e+03,4.986500e+03,7523.25,9999.00
CustomerId,9904.0,1.569097e+07,71958.601237,15565701.00,1.562853e+07,1.569078e+07,15753333.25,15815690.00
CreditScore,9904.0,6.506042e+02,96.599248,350.00,5.840000e+02,6.520000e+02,717.00,850.00
Age,9904.0,3.892023e+01,10.476533,18.00,3.200000e+01,3.700000e+01,44.00,92.00
Tenure,9904.0,5.013833e+00,2.892170,0.00,3.000000e+00,5.000000e+00,8.00,10.00
Balance,9904.0,7.656419e+04,62393.975288,0.00,0.000000e+00,9.726710e+04,127639.05,250898.09
NumOfProducts,9904.0,1.529584e+00,0.581735,1.00,1.000000e+00,1.000000e+00,2.00,4.00
HasCrCard,9904.0,7.050687e-01,0.456035,0.00,0.000000e+00,1.000000e+00,1.00,1.00
IsActiveMember,9904.0,5.145396e-01,0.499814,0.00,0.000000e+00,1.000000e+00,1.00,1.00
EstimatedSalary,9904.0,1.000495e+05,57465.622329,11.58,5.101484e+04,1.001343e+05,149216.32,199992.48


In [106]:
df['EstimatedSalary'].head()

0    101348.88
1    112542.58
2    113931.57
3     93826.63
4     79084.10
Name: EstimatedSalary, dtype: float64